In [1]:
import sys
sys.path.append("..")
import numpy as np
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time

In [2]:
# Класс для сохранения результатов
tests = Tests('test1')

# Создаем наблюдатель
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

In [3]:
# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [4]:
def target_func(seed=None):
    np.random.seed(seed)
    b = 0
    d = 10
    c = np.random.uniform(0, 180)
    v = np.random.uniform(3, 15)
    return [b, d, c, v]
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
precisions_1 = ['{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}',
'{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}','{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{:.0f}', '{0:.4f}', '{0:.0f}', '{0:.0f}']
precisions_2 = ['{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}', '{0:.2f}', '{0:.2f}', '{0:.0f}', '{0:.4f}']

In [5]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    tma.print_verbose()
    print('ММП v2')
    start_time = time.perf_counter()
    dict_results = tma.swarm(n=10, target_func=target_func, p0=p0, seeded=True)
    stop_time = time.perf_counter()
    print('Моделирование 10 результатов закончено за t = {:.1f} с'.format(stop_time - start_time))
    df = tests.get_df(dict_results)

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['СКО X', 'СКО Y', 'СКО VX', 'СКО VY', 'Кб', 'П0_апр', 'Д0_апр', 'К0_апр', 'V0_апр'], axis=1))

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}$, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('Nf = {:.0f}, Nf_max = {:.0f}'.format(df['Nf'].mean(), max(df['Nf'])))
    print('Iter = {:.0f}, Iter_max = {:.0f}'.format(df['Iter'].mean(), max(df['Iter'])))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))

    tests.save_df(df, name='../tests/tests8/ЛМ ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_______________________________________________')

СКОп = 0.0, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,360.0,10.000,98.786,11.58,70.56,15.807,70.56,15.807,0.00,"[1, 1, 1, 1]",0.0056,19,15
1,0.0,10.000,75.064,11.64,0.0,10.000,75.064,11.64,53.18,18.328,53.18,18.328,0.00,"[1, 1, 1, 1]",0.0032,12,10
2,0.0,10.000,78.479,3.31,360.0,10.000,78.479,3.31,31.77,9.618,31.77,9.618,0.00,"[1, 1, 1, 1]",0.0018,8,7
3,0.0,10.000,99.144,11.50,360.0,10.000,99.144,11.50,70.66,15.677,70.66,15.677,0.00,"[1, 1, 1, 1]",0.0042,19,15
4,0.0,10.000,174.065,9.57,0.0,10.000,174.065,9.57,149.70,4.673,149.70,4.673,0.00,"[1, 1, 1, 1]",0.0065,24,18
5,0.0,10.000,39.959,13.45,360.0,10.000,39.959,13.45,30.28,22.875,30.28,22.875,0.00,"[1, 1, 1, 1]",0.0027,9,8
6,0.0,10.000,160.715,6.98,360.0,10.000,160.715,6.98,97.62,3.973,97.62,3.973,0.00,"[1, 1, 1, 1]",0.0050,27,21
7,0.0,10.000,13.735,12.36,360.0,10.000,13.735,12.36,12.15,22.290,12.15,22.290,0.00,"[1, 1, 1, 1]",0.0017,8,7
8,0.0,10.000,157.217,14.62,360.0,10.000,157.217,14.62,137.83,11.865,137.83,11.865,0.00,"[1, 1, 1, 1]",0.0048,23,17
9,0.0,10.000,1.867,9.02,0.0,10.000,1.867,9.02,4.78,18.268,4.78,18.268,0.00,"[1, 1, 1, 1]",0.0015,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.00
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0037, t_max = 0.0065
_______________________________________________
СКОп = 0.1, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,360.0,10.026,98.804,11.63,70.56,15.807,70.58,15.864,0.01,"[1, 1, 1, 1]",0.0038,19,15
1,0.0,10.000,75.064,11.64,0.0,9.965,75.028,11.59,53.18,18.328,53.16,18.247,0.01,"[1, 1, 1, 1]",0.0024,12,10
2,0.0,10.000,78.479,3.31,0.0,10.008,78.788,3.30,31.77,9.618,31.78,9.600,0.01,"[1, 1, 1, 1]",0.0017,8,7
3,0.0,10.000,99.144,11.50,0.0,9.982,99.167,11.46,70.66,15.677,70.66,15.628,0.01,"[1, 1, 1, 1]",0.0035,19,15
4,0.0,10.000,174.065,9.57,360.0,10.016,174.042,9.59,149.70,4.673,149.68,4.682,0.01,"[1, 1, 1, 1]",0.0059,24,18
5,0.0,10.000,39.959,13.45,360.0,10.017,39.980,13.49,30.28,22.875,30.29,22.940,0.01,"[1, 1, 1, 1]",0.0019,9,8
6,0.0,10.000,160.715,6.98,360.0,10.057,160.576,7.04,97.62,3.973,97.61,4.013,0.01,"[1, 1, 1, 1]",0.0050,27,21
7,0.0,10.000,13.735,12.36,0.0,9.961,13.682,12.29,12.15,22.290,12.14,22.167,0.01,"[1, 1, 1, 1]",0.0017,8,7
8,0.0,10.000,157.217,14.62,360.0,9.989,157.214,14.60,137.83,11.865,137.83,11.852,0.01,"[1, 1, 1, 1]",0.0041,23,17
9,0.0,10.000,1.867,9.02,360.0,10.024,1.880,9.00,4.78,18.268,4.79,18.266,0.01,"[1, 1, 1, 1]",0.0017,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.01
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0032, t_max = 0.0059
_______________________________________________
СКОп = 0.2, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


...

Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.09
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0033, t_max = 0.0054
_______________________________________________
СКОп = 0.5, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,359.9,10.134,98.876,11.84,70.56,15.807,70.67,16.092,0.25,"[1, 1, 1, 1]",0.0044,19,15
1,0.0,10.000,75.064,11.64,0.0,9.830,74.882,11.36,53.18,18.328,53.08,17.928,0.25,"[1, 1, 1, 1]",0.0026,12,10
2,0.0,10.000,78.479,3.31,0.1,10.037,80.045,3.25,31.77,9.618,31.83,9.529,0.26,"[1, 1, 1, 1]",0.0019,8,7
3,0.0,10.000,99.144,11.50,0.1,9.910,99.260,11.29,70.66,15.677,70.65,15.433,0.26,"[1, 1, 1, 1]",0.0037,19,15
4,0.0,10.000,174.065,9.57,360.0,10.080,173.947,9.66,149.70,4.673,149.59,4.717,0.23,"[1, 1, 1, 1]",0.0045,24,18
5,0.0,10.000,39.959,13.45,359.9,10.087,40.065,13.67,30.28,22.875,30.33,23.201,0.24,"[1, 1, 1, 1]",0.0019,9,8
6,0.0,10.000,160.715,6.98,359.9,10.290,160.026,7.29,97.62,3.973,97.58,4.174,0.23,"[0, 1, 1, 1]",0.0050,27,21
7,0.0,10.000,13.735,12.36,0.1,9.808,13.469,12.01,12.15,22.290,12.08,21.688,0.24,"[1, 1, 1, 1]",0.0017,8,7
8,0.0,10.000,157.217,14.62,360.0,9.945,157.201,14.53,137.83,11.865,137.81,11.801,0.27,"[1, 1, 1, 1]",0.0042,23,17
9,0.0,10.000,1.867,9.02,360.0,10.123,1.929,8.91,4.78,18.268,4.80,18.257,0.26,"[1, 1, 1, 1]",0.0015,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 90$, Рэф_мин = 0
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.25
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0032, t_max = 0.0050
_______________________________________________
СКОп = 1.0, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,359.8,10.271,98.965,12.10,70.56,15.807,70.78,16.384,1.00,"[1, 1, 1, 1]",0.0037,19,15
1,0.0,10.000,75.064,11.64,0.1,9.668,74.699,11.09,53.18,18.328,52.99,17.545,0.99,"[1, 1, 1, 1]",0.0024,12,10
2,0.0,10.000,78.479,3.31,0.1,10.074,81.664,3.19,31.77,9.618,31.90,9.439,1.05,"[1, 1, 1, 1]",0.0016,8,7
3,0.0,10.000,99.144,11.50,0.2,9.821,99.377,11.09,70.66,15.677,70.64,15.192,1.05,"[1, 1, 1, 1]",0.0036,19,15
4,0.0,10.000,174.065,9.57,359.9,10.162,173.828,9.76,149.70,4.673,149.47,4.762,0.94,"[1, 1, 1, 1]",0.0044,24,18
5,0.0,10.000,39.959,13.45,359.8,10.176,40.172,13.90,30.28,22.875,30.37,23.533,0.96,"[1, 1, 1, 1]",0.0020,9,8
6,0.0,10.000,160.715,6.98,359.8,10.599,159.356,7.62,97.62,3.973,97.55,4.388,0.93,"[0, 0, 1, 1]",0.0050,27,21
7,0.0,10.000,13.735,12.36,0.1,9.627,13.201,11.67,12.15,22.290,12.00,21.111,0.95,"[0, 1, 1, 1]",0.0026,8,7
8,0.0,10.000,157.217,14.62,360.0,9.890,157.184,14.44,137.83,11.865,137.80,11.737,1.09,"[1, 1, 1, 1]",0.0044,23,17
9,0.0,10.000,1.867,9.02,360.0,10.246,1.994,8.80,4.78,18.268,4.81,18.246,1.05,"[1, 1, 1, 1]",0.0015,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 80$, Рэф_мин = 0
Вариант 1: Рэф = 90$, Рэф_мин = 0
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 1.00
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0031, t_max = 0.0050
_______________________________________________


In [6]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    tma.print_verbose()
    algorithm_name = 'Метод N пеленгов'
    print(algorithm_name)
    start_time = time.perf_counter()
    dict_results = tma.swarm(n=10, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True)
    stop_time = time.perf_counter()
    print('Моделирование 10 результатов закончено за t = {:.1f} с'.format(stop_time - start_time))
    df = tests.get_df(dict_results).dropna(axis=1)

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['Кб'], axis=1))

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}$, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))
    
    tests.save_df(df, name='../tests/tests8/N ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_______________________________________________________')

СКОп = 0.0, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,10.000,98.786,11.58,0.0,10.000,98.786,11.58,70.6,15.807,70.561,15.81,0.00,"[1, 1, 1, 1]",0.000
1,0.0,10.000,75.064,11.64,0.0,10.000,75.064,11.64,53.2,18.328,53.176,18.33,0.00,"[1, 1, 1, 1]",0.000
2,0.0,10.000,78.479,3.31,0.0,10.000,78.479,3.31,31.8,9.618,31.768,9.62,0.00,"[1, 1, 1, 1]",0.000
3,0.0,10.000,99.144,11.50,0.0,10.000,99.144,11.50,70.7,15.677,70.663,15.68,0.00,"[1, 1, 1, 1]",0.000
4,0.0,10.000,174.065,9.57,0.0,10.000,174.065,9.57,149.7,4.673,149.704,4.67,0.00,"[1, 1, 1, 1]",0.000
5,0.0,10.000,39.959,13.45,0.0,10.000,39.959,13.45,30.3,22.875,30.283,22.88,0.00,"[1, 1, 1, 1]",0.000
6,0.0,10.000,160.715,6.98,0.0,10.000,160.715,6.98,97.6,3.973,97.616,3.97,0.00,"[1, 1, 1, 1]",0.000
7,0.0,10.000,13.735,12.36,0.0,10.000,13.735,12.36,12.2,22.290,12.151,22.29,0.00,"[1, 1, 1, 1]",0.000
8,0.0,10.000,157.217,14.62,0.0,10.000,157.217,14.62,137.8,11.865,137.832,11.87,0.00,"[1, 1, 1, 1]",0.000
9,0.0,10.000,1.867,9.02,0.0,10.000,1.867,9.02,4.8,18.268,4.783,18.27,0.00,"[1, 1, 1, 1]",0.000


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.00
t = 0.0002, t_max = 0.0003
_______________________________________________________
СКОп = 0.1, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,10.000,98.786,11.58,359.8,10.151,98.690,11.91,70.6,15.807,70.597,16.18,0.01,"[1, 1, 1, 1]",0.000
1,0.0,10.000,75.064,11.64,359.8,9.999,74.867,11.77,53.2,18.328,53.162,18.47,0.01,"[1, 1, 1, 1]",0.000
2,0.0,10.000,78.479,3.31,0.0,9.995,79.076,3.27,31.8,9.618,31.763,9.55,0.01,"[1, 1, 1, 1]",0.000
3,0.0,10.000,99.144,11.50,359.8,10.118,98.991,11.77,70.7,15.677,70.670,15.99,0.01,"[1, 1, 1, 1]",0.000
4,0.0,10.000,174.065,9.57,360.0,9.999,174.061,9.56,149.7,4.673,149.699,4.67,0.01,"[1, 1, 1, 1]",0.000
5,0.0,10.000,39.959,13.45,360.0,9.987,39.936,13.49,30.3,22.875,30.284,22.90,0.01,"[1, 1, 1, 1]",0.000
6,0.0,10.000,160.715,6.98,0.0,9.908,160.941,6.89,97.6,3.973,97.643,3.91,0.01,"[1, 1, 1, 1]",0.000
7,0.0,10.000,13.735,12.36,359.8,9.774,13.617,12.63,12.2,22.290,12.144,22.39,0.01,"[1, 1, 1, 1]",0.000
8,0.0,10.000,157.217,14.62,360.0,9.957,157.219,14.55,137.8,11.865,137.837,11.81,0.01,"[1, 1, 1, 1]",0.000
9,0.0,10.000,1.867,9.02,360.0,10.020,1.845,8.96,4.8,18.268,4.776,18.22,0.01,"[1, 1, 1, 1]",0.000


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.01
t = 0.0003, t_max = 0.0003
_______________________________________________________
СКОп = 0.2, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


...

Показатели Монте-Карло:
Вариант 0: Рэф = 80$, Рэф_мин = 0
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.10
t = 0.0002, t_max = 0.0003
_______________________________________________________
СКОп = 0.5, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,10.000,98.786,11.58,359.1,9.110,97.028,10.69,70.6,15.807,70.309,14.61,0.38,"[0, 1, 1, 1]",0.000
1,0.0,10.000,75.064,11.64,359.2,9.003,72.950,10.76,53.2,18.328,52.766,16.81,0.36,"[0, 1, 1, 1]",0.000
2,0.0,10.000,78.479,3.31,0.2,9.734,81.388,2.94,31.8,9.618,31.535,8.97,0.27,"[0, 0, 0, 1]",0.000
3,0.0,10.000,99.144,11.50,359.1,8.864,96.977,10.24,70.7,15.677,70.228,14.06,0.46,"[0, 0, 0, 1]",0.000
4,0.0,10.000,174.065,9.57,360.0,9.759,174.290,9.26,149.7,4.673,149.886,4.53,0.25,"[1, 1, 1, 1]",0.000
5,0.0,10.000,39.959,13.45,359.8,9.282,39.082,12.50,30.3,22.875,30.046,21.15,0.27,"[0, 1, 1, 1]",0.000
6,0.0,10.000,160.715,6.98,0.2,9.160,162.543,6.12,97.6,3.973,97.675,3.43,0.25,"[0, 0, 0, 1]",0.000
7,0.0,10.000,13.735,12.36,359.2,8.518,12.614,12.73,12.2,22.290,11.889,21.26,0.33,"[1, 1, 1, 1]",0.000
8,0.0,10.000,157.217,14.62,360.0,9.350,157.383,13.49,137.8,11.865,138.008,11.04,0.30,"[0, 1, 1, 1]",0.000
9,0.0,10.000,1.867,9.02,360.0,9.851,1.056,8.16,4.8,18.268,4.536,17.08,0.27,"[0, 1, 1, 1]",0.001


Показатели Монте-Карло:
Вариант 0: Рэф = 20$, Рэф_мин = 0
Вариант 1: Рэф = 70$, Рэф_мин = 0
Вариант 2: Рэф = 70$, Рэф_мин = 0
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.32
t = 0.0003, t_max = 0.0006
_______________________________________________________
СКОп = 1.0, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,10.000,98.786,11.58,358.2,5.751,89.933,5.83,70.6,15.807,68.542,8.58,3.09,"[0, 0, 0, 0]",0.000
1,0.0,10.000,75.064,11.64,358.4,6.611,67.437,7.56,53.2,18.328,51.393,11.97,2.17,"[0, 0, 0, 0]",0.000
2,0.0,10.000,78.479,3.31,0.4,8.991,84.520,2.27,31.8,9.618,30.747,7.72,1.22,"[0, 0, 0, 0]",0.000
3,0.0,10.000,99.144,11.50,358.2,5.511,89.020,5.42,70.7,15.677,68.188,8.08,3.68,"[0, 0, 0, 0]",0.000
4,0.0,10.000,174.065,9.57,359.9,9.014,175.156,8.33,149.7,4.673,150.621,4.09,1.12,"[0, 0, 0, 1]",0.000
5,0.0,10.000,39.959,13.45,359.6,7.521,36.149,9.69,30.3,22.875,29.151,16.37,1.42,"[0, 0, 0, 0]",0.000
6,0.0,10.000,160.715,6.98,0.3,7.765,166.411,4.70,97.6,3.973,97.541,2.56,1.14,"[0, 0, 0, 0]",0.000
7,0.0,10.000,13.735,12.36,358.3,6.653,10.344,11.35,12.2,22.290,11.099,17.76,1.64,"[0, 0, 0, 0]",0.000
8,0.0,10.000,157.217,14.62,359.9,7.955,157.968,11.04,137.8,11.865,138.566,9.26,1.50,"[0, 0, 0, 0]",0.000
9,0.0,10.000,1.867,9.02,360.0,9.174,358.143,6.37,4.8,18.268,3.719,14.23,1.26,"[0, 0, 0, 0]",0.000


Показатели Монте-Карло:
Вариант 0: Рэф = 0$, Рэф_мин = 0
Вариант 1: Рэф = 0$, Рэф_мин = 0
Вариант 2: Рэф = 0$, Рэф_мин = 0
Вариант 3: Рэф = 10$, Рэф_мин = 0
Ка = 1.82
t = 0.0002, t_max = 0.0003
_______________________________________________________
